# Libraries

In [9]:
import glob

import numpy as np
import pandas as pd

In [16]:
file_paths = glob.glob('Kaggle_HDB/*.csv')

df = []
for file in file_paths:
    print(file)
    df_temp = pd.read_csv(file, parse_dates=['month'])
    df.append(df_temp)

# 3. Concatenate all DataFrames into one
df_combined = pd.concat(df, ignore_index=True).reset_index(drop=True)
df_combined.head()

Kaggle_HDB\resale-flat-prices-based-on-approval-date-1990-1999.csv
Kaggle_HDB\resale-flat-prices-based-on-approval-date-2000-feb-2012.csv
Kaggle_HDB\resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv
Kaggle_HDB\resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv
Kaggle_HDB\resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN
1,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN
2,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN
3,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN
4,1990-01-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN
